In [1]:
from nbodykit.lab import *
from nbodykit import style, setup_logging

import corner
from tqdm import tqdm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

import sys, pickle, time, os, h5py

from nbodykit.source.catalog import BinaryCatalog
# nbodykit cosmology parameters initialization
from nbodykit.lab import FFTPower

In [2]:
COSMOPAR = {
#                   | Om   | Ob   |   h   |  n_s  | s_8 | Mnu | w |

    'fiducial' :    [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    'zeldovich':    [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    
    'Mnu_p' :       [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.1, -1],
    'Mnu_pp' :      [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.2, -1],
    'Mnu_ppp' :     [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.4, -1],
    
    'h_m' :         [0.3175, 0.049, 0.6511, 0.9624, 0.834, 0, -1],
    'h_p' :         [0.3175, 0.049, 0.6911, 0.9624, 0.834, 0, -1],
    
    'ns_m' :        [0.3175, 0.049, 0.6711, 0.9424, 0.834, 0, -1],
    'ns_p' :        [0.3175, 0.049, 0.6711, 0.9824, 0.834, 0, -1],
    
    'Ob_m' :        [0.3175, 0.048, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob_p' :        [0.3175, 0.050, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob2_m' :       [0.3175, 0.047, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob2_p' :       [0.3175, 0.051, 0.6711, 0.9624, 0.834, 0, -1],
    
    'Om_m' :        [0.3075, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    'Om_p' :        [0.3275, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    
    's8_m' :        [0.3175, 0.049, 0.6711, 0.9624, 0.819, 0, -1],
    's8_p' :        [0.3175, 0.049, 0.6711, 0.9624, 0.849, 0, -1]
}

folders_ordered = {
    0 :'fiducial'  ,    
    1 :'h_m'       , 
    2 :'h_p'       ,    
    3 :'Mnu_p'     , 
    4 :'Mnu_pp'    ,
    5 :'Mnu_ppp'   ,
    6 :'ns_m'      , 
    7 :'ns_p'      ,    
    8 :'Ob2_m'     , 
    9 :'Ob2_p'     ,    
    10 :'Om_m'      , 
    11 :'Om_p'      ,
    12 :'s8_m'      , 
    13 :'s8_p'      ,
    14 :'zeldovich' ,}

order_folders = {
    'fiducial'  : 0,
    'h_m'       : 1,  'h_p'       : 2,
    'Mnu_p'     : 3,  'Mnu_pp'    : 4, 'Mnu_ppp'   : 5, 
    'ns_m'      : 6,  'ns_p'      : 7,
    'Ob2_m'     : 8,  'Ob2_p'     : 9,
    'Om_m'      : 10, 'Om_p'      : 11,
    's8_m'      : 12, 's8_p'      : 13,
    'zeldovich' : 14
}

order_dimension = {
    'Om'  : 0, 'Om ' : 0,
    'Ob'  : 1, 'Ob ' : 1, 'Ob2' : 1,
    'h'   : 2, 'h  ' : 2, # 4
    'ns'  : 3, 'ns ' : 3,
    's8'  : 4, 's8 ' : 4, # 2
    'Mnu' : 5
}

"""Om, Ob, s8, ns, h, Mnu"""

cosmological_pars = {
    'Om'  : 0, 'Ob'  : 1, 'h'   : 2,
    'ns'  : 3, 's8'  : 4, 'Mnu' : 5
}

VarCosmoPar = {
    'd_h'  : 0.02, 'd_ns' : 0.02,  'd_Ob' : 0.001, 'd_Ob2': 0.002,
    'd_Om' : 0.01, 'd_s8' : 0.015
}

fiducial_vals = {
    'Ob'  : 0.3175, 'Ob2' : 0.3175,
    'Om'  : 0.049,
    'h'   : 0.6711,
    'n_s' : 0.9624, 'ns'  : 0.9624,
    's_8' : 0.834,  's8'  : 0.834,
    'Mnu' : 0
}

In [3]:
def correlation_matrix(m):
    """Calculates the correlation matrix of matrix m_ij."""
    avg = np.average(m, axis=0)
    dim = len(avg)
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # VARIATION
    sigma = []
    for i in range(dim):
        sigma.append(np.sum((avg[i] - m[i])**2))
    sigma = np.sqrt(sigma)
    
    sigma_bis = []
    for i in range(dim):
        cum = 0
        for j in range(len(m[i])):
            cum += (avg[i] - m[i][j])**2
        sigma_bis.append(cum)
    sigma_bis = np.sqrt(sigma_bis)        
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # CORRELATION MATRIX
    CORR = np.zeros((dim, dim))
    for i in range(dim):
        for j in range(dim):
            cum = 0
            for k in range(len(m[i])):
                cum += (avg[i] - m[i][k])*(avg[j] - m[j][k])
            CORR[i, j] = cum  / (sigma_bis[i] * sigma_bis[j])

    if np.linalg.det(CORR) == 0:
        print("¶ WARNING: correlation matrix is singular")
    
    return CORR

def Hartlap(mat, Nr = 350):
    """Calculates inverse matrix using Hartlap correction.
    Arguments:
    - `mat`: input matrix to invert
    - `Nr`: nuber of realization used o calculated the matrix
    """
    return (Nr-len(mat)-2)/(Nr-1)*np.linalg.inv(mat)

def info_name(name):
    """Obtain realization information from namefile"""
    print("TYPE: ", type(name))
    info = name.split('_')[-3:]
    print("TYPE: ", info, "\n", type(info))
    info[2] = info[2].replace(".wst", "")
    N_hgrid = info[0]
    N_WSTgrid = info[1]
    n_realiz = info[2].replace(".wst", "")
    
    return [int(N_hgrid), int(N_WSTgrid), int(n_realiz)]

def cosmo_parser(name):
    """Obtain cosmology from .wst file"""
    info = name.split('_')
    if info[0] == "fiducial":
        return info[0]
    elif info[0] == "zeldovich":
        return info[0]
    else:
        return info[0] + "_" + info[1]
    
def covariation_matrix(m):
    """Calculates the correlation matrix of matrix m_ij.
    """
    print('Shape: ' + str(np.shape(m)))
    avg = np.average(m, axis=0)    # mean values
    dim = len(avg)                 # number of variables
    N = len(m[0])                  # number of values per varaible
    print('len matr ' + str(len(m[0])))
    print('   len avg ' + str(dim))
    COV = np.zeros((dim, dim))
    for i in range(dim):
        for j in range(dim):
            cum = 0.
            for k in range(len(m[i])):
                cum += (avg[i] -
                         m[i][k])*\
                    (avg[j] - 
                     m[j][k])
            COV[i, j] = cum/N

    if np.linalg.det(COV) == 0:
        print("¶ WARNING: correlation matrix is singular")
    
    return COV

def error_message(a = "ERROR >.<"):
    N = len(a)
    print("")
    print(" "*6 + "_"*N)
    print(" "*5+"/" + " "*N + "\\")
    print(" === | " + a + "| ===")
    print(" "*5 + "\\" + "_"*N + "/" )
    print("")

In [4]:
root_WST = "/home/fuffolo97/TESI/cubes/WST_3D_files/"
root_Pk  = "/home/fuffolo97/TESI/cubes/Pk_3D_files/"
files_to_read_wst = os.listdir(root_WST)
files_to_read_pk = os.listdir(root_Pk)


In [5]:
folders = [
    'fiducial', 'zeldovich', 'h_m', 'h_p', 'Mnu_p', 'Mnu_pp' ,'Mnu_ppp',
    'ns_m', 'ns_p', 'Ob2_m', 'Ob2_p',
    'Om_m', 'Om_p', 's8_m', 's8_p'
]
cosmologies = [
    'fiducial', 'zeldovich', 'h_m', 'h_p', 'Mnu_p', 'Mnu_pp' ,'Mnu_ppp',
    'ns_m', 'ns_p', 'Ob2_m', 'Ob2_p',
    'Om_m', 'Om_p', 's8_m', 's8_p'
]
parameter_name = ['Om ', 'Ob ', 'h  ', 'ns ', 's8 ', 'Mnu']

In [6]:
fiducial_wst_real = []
zeldovich_wst_real = []
fiducial_pk_real = []
zeldovich_pk_real = []
coeffs_wst = np.zeros((len(cosmologies), 75))
coeffs_pk  = np.zeros((len(cosmologies), 159))

In [7]:
# for i in range(len(files_to_read_wst)):
#     coeffs_cosm = []
#     file_reading = files_to_read_wst[i]

#     cosmo = cosmo_parser(file_reading)
#     assert cosmo in file_reading, f"{cosmo} not in actual cosmology"
#     # print("COSMO KEY: ", cosmo)
#     index = order_folders[cosmo]
    
#     if ("fiducial" not in file_reading) and ("zeldovich" not in file_reading):
#         # print("A, ", file_reading)
#         with open(root_WST + file_reading, 'rb') as Ff:
#             while True:
#                 try:
#                     coeffs_cosm.append(pickle.load(Ff))
#                 except EOFError:
#                     break
#                 except RuntimeWarning:
#                     print("problem here: ", file_reading)

#     elif "fiducial" in file_reading:
#         # print("B, ", file_reading)
#         with open(root_WST + file_reading, 'rb') as Ff:
#             while True:
#                 try:
#                     fiducial_wst_real.append(pickle.load(Ff))
#                 except EOFError:
#                     break
#                 except RuntimeWarning:
#                     print("problem here: ", file_reading)
#         fiducial_coeffs_N_avg = np.average(fiducial_wst_real, axis=0)

#         print(np.shape(fiducial_coeffs_N_avg))
        
#         if len(fiducial_wst_real) == 0:
#             print(cosmo, " has problem!")

#     elif "zeldovich" in file_reading:
#         # print("C, ", file_reading)
#         with open(root_WST + file_reading, 'rb') as Ff:
#             while True:
#                 try:
#                     zeldovich_wst_real.append(pickle.load(Ff))
#                 except EOFError:
#                     break
#                 except RuntimeWarning:
#                     print("problem here: ", file_reading)
#         # print(zeldovich_coeffs)
#         zeldovich_coeffs_N_avg = np.average(zeldovich_wst_real, axis=0)
#         # print(zeldovich_coeffs_avg)
#         print(np.shape(zeldovich_coeffs_N_avg))
#     else:
#         assert False, "\n   ERROR in reading WST files, cosmology can't be propely interpreted.\n"

#     # USING average  value per coefficient in cosmology
#     if len(coeffs_cosm) == 0 and cosmo != "fiducial" and cosmo != "zeldovich":
#         print(cosmo, " has problem!")
#         print(cosmo, " : ", np.shape(coeffs_cosm))
#     if cosmo != "fiducial" and cosmo != "zeldovich":
#         coeffs_wst[index] = np.average(coeffs_cosm, axis=0)

In [8]:
# for i in range(len(files_to_read_pk)):
#     coeffs_cosm = []
#     file_reading = files_to_read_pk[i]

#     cosmo = cosmo_parser(file_reading)
#     assert cosmo in file_reading, f"{cosmo} not in actual cosmology"
#     # print("COSMO KEY: ", cosmo)
#     index = order_folders[cosmo]
    
#     if ("fiducial" not in file_reading) and ("zeldovich" not in file_reading):
#         # print("A, ", file_reading)
#         with open(root_Pk + file_reading, 'rb') as Ff:
#             while True:
#                 try:
#                     a = pickle.load(Ff)
#                     #a.power
#                     a = a['power'].real
#                     coeffs_cosm.append(a)
#                 except EOFError:
#                     break
#                 except RuntimeWarning:
#                     print("problem here: ", file_reading)

#     elif "fiducial" in file_reading:
#         # print("B, ", file_reading)
#         with open(root_Pk + file_reading, 'rb') as Ff:
#             while True:
#                 try:
#                     a = pickle.load(Ff)
#                     #a.power
#                     a = a['power'].real
#                     fiducial_pk_real.append(a)
#                 except EOFError:
#                     break
#                 except RuntimeWarning:
#                     print("problem here: ", file_reading)
#         fiducial_pk_avg = np.average(fiducial_pk_real, axis=0)

#         print(np.shape(fiducial_pk_avg))
        
#         if len(fiducial_pk_real) == 0:
#             print(cosmo, " has problem!")

#     elif "zeldovich" in file_reading:
#         # print("C, ", file_reading)
#         with open(root_Pk + file_reading, 'rb') as Ff:
#             while True:
#                 try:
#                     a = pickle.load(Ff)
#                     #a.power
#                     a = a['power'].real
#                     zeldovich_pk_real.append(a)
#                 except EOFError:
#                     break
#                 except RuntimeWarning:
#                     print("problem here: ", file_reading)
#         # print(zeldovich_coeffs)
#         zeldovich_pk_avg = np.average(zeldovich_pk_real, axis=0)
#         # print(zeldovich_coeffs_avg)
#         print(np.shape(zeldovich_pk_avg))
#     else:
#         assert False, "\n   ERROR in reading WST files, cosmology can't be propely interpreted.\n"

#     # USING average  value per coefficient in cosmology
#     if len(coeffs_cosm) == 0 and cosmo != "fiducial" and cosmo != "zeldovich":
#         print(cosmo, " has problem!")
#         print(cosmo, " : ", np.shape(coeffs_cosm))
#     if cosmo != "fiducial" and cosmo != "zeldovich":
#         coeffs_pk[index] = np.average(coeffs_cosm, axis=0)

In [9]:
to_do_list = ['ns_p', 'Ob2_m', 'Om_p', 's8_p']

In [10]:
files_to_read_wst

['Ob2_p_coefficients_real.wst',
 'Mnu_pp_coefficients_real.wst',
 'zeldovich_coefficients_real.wst',
 'ns_p_coefficients_real.wst',
 'Om_m_coefficients_real.wst',
 'Om_p_wst_real.hdf5',
 'h_m_coefficients_real.wst',
 'Mnu_ppp_coefficients_real.wst',
 's8_p_wst_real.hdf5',
 'Mnu_p_coefficients_real.wst',
 'Om_p_coefficients_real.wst',
 'Ob2_m_coefficients_real.wst',
 'h_p_coefficients_real.wst',
 'ns_p_wst_real.hdf5',
 'fiducial_coefficients_real.wst',
 's8_m_coefficients_real.wst',
 'ns_m_coefficients_real.wst',
 's8_p_coefficients_real.wst',
 'Ob2_m_wst_real.hdf5']

In [11]:
every_mean_wst_pickle = np.zeros((len(order_folders) - len(to_do_list), 75))
every_mean_wst_h5py   = np.zeros((len(to_do_list), 75))
every_mean_wst = np.zeros((len(order_folders), 75))

for FC in range(len(files_to_read_wst)):
    every_wst = []

    cosmo = cosmo_parser(files_to_read_wst[FC])
    assert cosmo in files_to_read_wst[FC]
    index = order_folders[cosmo]
    if cosmo not in to_do_list:
        print(f'\n\n{cosmo} in `if`')
        # reading data from files
        with open('WST_3D_files/'+files_to_read_wst[FC], "rb") as file:
            while True:
                try:
                    every_wst.append(pickle.load(file))
                except EOFError:
                    break
        every_mean_wst[index] = np.mean(every_wst, axis=0)
    elif cosmo in to_do_list and '.hdf5' in files_to_read_wst[FC]:
        print(f'\n\n{cosmo} in `elif`')
        with h5py.File('WST_3D_files/'+files_to_read_wst[FC], 'r') as fh:
            for i in range(350):
                dat = np.array(fh[f'{cosmo}_{i}_wst'])
                every_wst.append(dat)
                if i == 298: print(dat)
        every_mean_wst[index] = np.mean(every_wst, axis=0)
    else:
        Boo = '.hdf5' in files_to_read_wst[FC]
        print(f'\n\n{cosmo} in `continue`')
        print(f'    {cosmo in to_do_list}  -  {cosmo}')
        print('    ' + str(Boo) + "  - " + files_to_read_wst[FC])
        continue
    print(f'DOING WITH {files_to_read_wst[FC]}\n')
    # every_mean_wst[index] = np.mean(every_wst, axis=0)
    # print(np.mean(every_wst, axis=0))
    # print(every_mean_wst[index])



Ob2_p in `if`
DOING WITH Ob2_p_coefficients_real.wst



Mnu_pp in `if`
DOING WITH Mnu_pp_coefficients_real.wst



zeldovich in `if`
DOING WITH zeldovich_coefficients_real.wst



ns_p in `continue`
    True  -  ns_p
    False  - ns_p_coefficients_real.wst


Om_m in `if`
DOING WITH Om_m_coefficients_real.wst



Om_p in `elif`
[1.55417015e+14 9.05736712e+13 6.73764438e+13 5.42422577e+13
 4.51214165e+13 1.16077648e+14 7.95260758e+13 6.22746392e+13
 5.19313262e+13 4.48793590e+13 7.57467143e+13 5.80276590e+13
 4.65286558e+13 3.92005692e+13 3.41193251e+13 4.34892114e+13
 3.60264122e+13 2.99298172e+13 2.57102782e+13 2.25958691e+13
 2.25577345e+13 1.98952616e+13 1.67750392e+13 1.45929567e+13
 1.30383159e+13 1.63973379e+14 3.29303133e+13 1.71272852e+13
 9.03573668e+12 4.65782073e+12 1.65768692e+14 2.70440543e+13
 1.60649068e+13 1.03460687e+13 6.86950606e+12 1.66387503e+14
 1.74101784e+13 1.09424232e+13 7.52593509e+12 5.41130123e+12
 1.66537038e+14 9.68078288e+12 6.20843604e+12 4.37758722e+12
 

In [12]:
for i in to_do_list:
    every_wst = []
    cosmo = cosmo_parser(i)
    assert cosmo in i
    index = order_folders[cosmo]

    with h5py.File('WST_3D_files/'+i+'_wst_real.hdf5', 'r') as fh:
        for i in range(350):
            dat = np.array(fh[f'{cosmo}_{i}_wst'])
            print(dat)
            every_wst.append(dat)
    print('\n')
    print(np.mean(every_wst, axis=0))
    print('\n\n==============================================================================0\n')

[1.64962027e+14 9.62447142e+13 7.15882129e+13 5.76289275e+13
 4.79354170e+13 1.23070115e+14 8.44162232e+13 6.61147552e+13
 5.51299066e+13 4.76351845e+13 8.02944975e+13 6.14217149e+13
 4.92361419e+13 4.14764614e+13 3.60932190e+13 4.61558535e+13
 3.83249453e+13 3.17275358e+13 2.72012212e+13 2.38801755e+13
 2.37487323e+13 2.12480756e+13 1.81475329e+13 1.57905595e+13
 1.40135786e+13 1.74033669e+14 3.48924401e+13 1.81406458e+13
 9.56739512e+12 4.93181207e+12 1.75919344e+14 2.85786956e+13
 1.69556228e+13 1.09103934e+13 7.24046656e+12 1.76567716e+14
 1.84875357e+13 1.15958077e+13 7.95672786e+12 5.70808651e+12
 1.76725724e+14 1.02570226e+13 6.69176404e+12 4.73522766e+12
 3.49756536e+12 1.29381922e+14 2.19278695e+13 1.21694344e+13
 7.00437076e+12 4.02231211e+12 1.30096598e+14 1.54403833e+13
 9.81461998e+12 6.75642396e+12 4.84404154e+12 1.30273019e+14
 8.70532828e+12 5.90147853e+12 4.32156862e+12 3.31530502e+12
 8.41618974e+13 1.05318753e+13 5.89071909e+12 3.40825158e+12
 1.98635107e+12 8.442667

In [59]:
err_files = {
    'ns_p'  : 298,
    'Ob2_m' : 69,
    'Om_p'  : 187,
    's8_p'  : 331
}

every_wst = []
cosm = 's8_p'
with h5py.File(f'WST_3D_files/{cosm}_wst_real.hdf5', 'r') as fh:
    for i in range(350):
        dat = np.array(fh[f'{cosm}_{i}_wst'])
        every_wst.append(dat)
a, b = np.shape(every_wst)

In [68]:
np.mean(a=(every_wst[:335]), axis=0)

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)

In [65]:
np.mean(every_wst[-10:], axis=0)

array([1.6160519e+14, 9.3816967e+13, 6.9814828e+13, 5.6269428e+13,
       4.6857649e+13, 1.2110061e+14, 8.2741941e+13, 6.4788214e+13,
       5.4053782e+13, 4.6743119e+13, 7.9175013e+13, 6.0644481e+13,
       4.8623115e+13, 4.0961665e+13, 3.5643577e+13, 4.5341202e+13,
       3.7816306e+13, 3.1383538e+13, 2.6920845e+13, 2.3631678e+13,
       2.3148846e+13, 2.0724276e+13, 1.7673421e+13, 1.5401870e+13,
       1.3733915e+13, 1.7054372e+14, 3.4513623e+13, 1.7965032e+13,
       9.4909822e+12, 4.8984306e+12, 1.7245250e+14, 2.8463128e+13,
       1.6925009e+13, 1.0910607e+13, 7.2484829e+12, 1.7311520e+14,
       1.8391140e+13, 1.1567683e+13, 7.9554507e+12, 5.7157133e+12,
       1.7327437e+14, 1.0147629e+13, 6.5822969e+12, 4.6521469e+12,
       3.4496237e+12, 1.2735385e+14, 2.1964043e+13, 1.2209118e+13,
       7.0385779e+12, 4.0510551e+12, 1.2807680e+14, 1.5442507e+13,
       9.8376478e+12, 6.7841326e+12, 4.8751953e+12, 1.2825214e+14,
       8.7091357e+12, 5.8501125e+12, 4.2635147e+12, 3.2774942e

In [79]:
'nan' in str(every_wst[331])

True

In [71]:
with open('NaN_arary.dat', 'wb') as f:
    f.write(every_wst[331])

In [74]:
with open('NaN_arary.dat', 'rb') as f:
    DD = np.array(f.read())

In [75]:
DD

array(b'\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\xff\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f\x00\x00\xc0\x7f

In [18]:
def mean_coeffs(M):
    X, Y = np.shape(M)
    r = np.zeros(Y)
    M *= 1e-14
    for j in range(Y):  # indice coeff
        c = 0
        for i in range(X): # indice realiz
            c += (M[i][j]/X)
        r[j] = c
    return r*1e14

In [19]:
every_wst = []
with h5py.File('WST_3D_files/ns_p_wst_real.hdf5', 'r') as fh:
    for i in range(350):
        dat = np.array(fh[f'ns_p_{i}_wst']).astype(np.float256)
        # print(dat)
        every_wst.append(dat)
final_array = mean_coeffs(np.array(every_wst))
print(final_array)

AttributeError: module 'numpy' has no attribute 'float256'

In [ ]:
final_array[0]

nan

In [ ]:
break

In [ ]:
for i in range(len(every_mean_wst)):
    print(folders_ordered[i])
    print(every_mean_wst[i])

fiducial
[2.80201854e+14 1.62984245e+14 1.21106829e+14 9.74587387e+13
 8.10559314e+13 2.09577325e+14 1.43416910e+14 1.12201298e+14
 9.35328701e+13 8.08149435e+13 1.36786311e+14 1.04963984e+14
 8.40893192e+13 7.07815140e+13 6.15668253e+13 7.79799295e+13
 6.53737098e+13 5.42821036e+13 4.65768190e+13 4.08970435e+13
 3.92513496e+13 3.55372431e+13 3.04631691e+13 2.66207630e+13
 2.37575906e+13 2.95626793e+14 5.93762913e+13 3.08429801e+13
 1.62725269e+13 8.39247095e+12 2.98888553e+14 4.88996371e+13
 2.90196575e+13 1.86752162e+13 1.23956500e+13 3.00016451e+14
 3.15424936e+13 1.98280898e+13 1.36307802e+13 9.79088232e+12
 3.00285860e+14 1.72734577e+13 1.12446344e+13 7.95872382e+12
 5.90127615e+12 2.20368397e+14 3.76870210e+13 2.09221614e+13
 1.20436021e+13 6.92383069e+12 2.21606991e+14 2.64524729e+13
 1.68401777e+13 1.16076336e+13 8.33859302e+12 2.21905995e+14
 1.48339111e+13 9.99705962e+12 7.29196999e+12 5.60096084e+12
 1.43413119e+14 1.79900892e+13 1.01291729e+13 5.88493357e+12
 3.43911196e+12

In [ ]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [81]:
every_mean_pk = np.zeros((len(order_folders), 159))
real_power = np.zeros((len(order_folders), 159))

for FC in tqdm(range(len(files_to_read_pk))):
    every_pk = []
    every_power = []

    cosmo = cosmo_parser(files_to_read_pk[FC])
    assert cosmo in files_to_read_pk[FC]
    index = order_folders[cosmo]

    if cosmo not in to_do_list:
    # reading data from files
        with open('Pk_3D_files/'+files_to_read_pk[FC], "rb") as file:
            while True:
                try:
                    every_pk.append(pickle.load(file))
                except EOFError:
                    break
        for i in range(len(every_pk)):
            every_power.append(every_pk[i]["power"].real)#  - every_pk[i].attrs['shotnoise'] )
        real_power[index] += np.mean(every_power, axis=0)
    elif cosmo in to_do_list and '.hdf5' in files_to_read_pk[FC]:
        with h5py.File('Pk_3D_files/'+files_to_read_pk[FC], 'r') as f:
            for i in range(350):
                every_power.append(np.array(f[f'{cosmo}_{i}_pk']))
        real_power[index] += np.mean(every_power, axis=0)
    else: continue

    # for i in range(len(every_pk)):
    #     real_power.append(every_pk[i]["power"].real  - every_pk[i].attrs['shotnoise'] )

    every_mean_pk[index] += np.mean(real_power, axis=0)

100%|██████████| 19/19 [00:00<00:00, 43.77it/s]


In [ ]:
deriavates_wst = np.zeros((len(cosmological_pars),  75))
deriavates_pk  = np.zeros((len(cosmological_pars), 159))

for i in cosmological_pars:
    if "Mnu" not in i and "Ob" not in i:
        ind = order_dimension[i]
        deriavates_wst[ind]=(every_mean_wst[order_folders[i+"_p"]]-every_mean_wst[order_folders[i+"_m"]])\
                    /  (2 * VarCosmoPar['d_'+i] * fiducial_vals[i] )
        deriavates_pk[ind]=(real_power[order_folders[i+"_p"]]-real_power[order_folders[i+"_m"]])\
                    /  (2 * VarCosmoPar['d_'+i] * fiducial_vals[i] )

    elif "Mnu" in i:
        deriavates_wst[order_dimension['Mnu']] = (every_mean_wst[order_folders["Mnu_p"]] - every_mean_wst[order_folders["zeldovich"]]) / (0.1)
        deriavates_pk[order_dimension['Mnu']]  = (real_power[order_folders["Mnu_p"]]  - real_power[order_folders["zeldovich"]])  / (0.1)

    elif "Ob" in i:
        deriavates_wst[order_dimension['Ob']] = \
            (every_mean_wst[order_folders[i+"2_p"]]-\
             every_mean_wst[order_folders[i+"2_m"]]) \
                / (2 * VarCosmoPar['d_'+i+"2"] * fiducial_vals[i] )
        deriavates_pk[order_dimension['Ob']] = \
            (real_power[order_folders[i+"2_p"]]-\
             real_power[order_folders[i+"2_m"]]) \
                / (2 * VarCosmoPar['d_'+i+"2"] * fiducial_vals[i] )


In [ ]:
deriavates_wst

array([[            nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             na

In [ ]:
# create transposed array of Pk coefficients
fiducial_wst = every_mean_wst[order_folders['fiducial']].transpose()
fiducial_pk = every_mean_pk[order_folders['fiducial']].transpose()

In [ ]:
fiducial_wst = []
fiducial_pk = []

with open("./WST_3D_files/fiducial_coefficients_real.wst", "rb") as f:
    while True:
        try:
            fiducial_wst.append(pickle.load(f))
        except EOFError:
            break

with open("./Pk_3D_files/fiducial_coefficients_real.pk", "rb") as f:
    while True:
        try:
            a = pickle.load(f)
            a = a['power'].real
            fiducial_pk.append(a)
        except EOFError:
            break

# create transposed array of Pk coefficients
fiducial_wst = np.array(fiducial_wst).transpose()
fiducial_pk = np.array(fiducial_pk).transpose()

# cov_wst = covariation_matrix(fiducial_wst)
# cov_pk  = covariation_matrix(fiducial_pk)

cov_wst = np.cov(fiducial_wst)
cov_pk  = np.cov(fiducial_pk)

# inverting with Hartlap factor
Hart_wst = Hartlap(cov_wst, 1000)
Hart_pk = Hartlap(cov_pk, 1000)

# initialize empty Fisher matrix
Fish_wst, Fish_pk  = np.zeros((6,6)), np.zeros((6,6))

# create Fisher matrix
for a in range(6):
    for b in range(6):
        Fish_wst[a, b]  = np.sum(deriavates_wst[a]  * (Hart_wst  * deriavates_wst[b]))
        Fish_pk[a, b]   = np.sum(deriavates_pk[a]   * (Hart_pk   * deriavates_pk[b]))

# GETTING CONATRAINS:
# create inverse of Fisher matrix
inverse_wst = np.linalg.inv(Fish_wst)
inverse_pk = np.linalg.inv(Fish_pk)

# initialize empty array containing diagonal elements
diagonal_wst, diagonal_pk = [], []

# inizialize empty matrix containing all correletions
constrains_wst, constrains_pk = np.zeros((6,6)), np.zeros((6,6))

# `for loop`: assegnate diaconal elements
for i in range(len(inverse_wst)):
    diagonal_wst.append(inverse_wst[i, i])
    diagonal_pk.append(inverse_pk[i, i])
    # internal `for loop`: assegnate all the correlations
    for j in range(6):
        constrains_wst[i, j] += inverse_wst[i, j]
        constrains_pk[i, j] += inverse_pk[i, j]

# after checking symmetric elements are almost equal, set them as equal
# this is needed because exact symm. matrix is needed
for i in range(6):
    for j in range(6):
        assert (np.abs(constrains_wst[i, j] - constrains_wst[j, i]) < 1e-5), f'{np.abs(constrains_wst[i, j] - constrains_wst[j, i])}'
        assert (np.abs(constrains_pk[i, j] - constrains_pk[j, i]) < 1e-5), f'{np.abs(constrains_pk[i, j] - constrains_pk[j, i])}'
        constrains_wst[i, j] = np.abs(constrains_wst[j, i])
        constrains_pk[i, j] = constrains_pk[j, i]

# # create correlation graphs, maybe useful to confront with wst
# sns.heatmap(corr.transpose())
# plt.gca().invert_yaxis()
# plt.savefig('correlation_matrix_pk.pdf', format='pdf')

# sns.heatmap(corr_rsd.transpose())
# plt.gca().invert_yaxis()
# plt.savefig('correlation_matrix_rsd_pk.pdf', format='pdf')

# # save constrains in a file
# with open("./ZZ_results/constrains_pk.res", "ab") as f:
#     pickle.dump(constrains, f)
# with open("./ZZ_results/constrains_rsd_pk.res", "ab") as f:
#     pickle.dump(constrains_rsd, f)

AssertionError: nan